# Roulette outcomes

In this exercise, we'll explore the fortunes of someone playing a roulette game using a wheel that contains the numbers 1-36. We'll then look at the effect of including the zero and double-zero that push the odds in the house's favor.

In [2]:
# Loading modules and setting up environment
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import copy
from collections import Counter
from collections import OrderedDict

# Load Widgets for GUI
from ipywidgets import widgets
from ipywidgets import interact, IntSlider
from IPython.display import display

mpl.get_backend()

'module://ipykernel.pylab.backend_inline'

Roulette has multiple betting strategies, but we'll restrict ourselve to two of them

+ Betting on even/odd - pays 2x (original bet + like amount) on win
+ Betting on a single number - pays 36x (original bet + 35x) on a win

So that we can easily handle the zero and double-zero later in this exercise, we'll work with strings rather than integers and write the smaller numbers with leading zero.

In [3]:
bins_nozero = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
               '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
               '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',
               '31', '32', '33', '34', '35', '36']
bins_zero = copy.copy(bins_nozero)
bins_zero.append('0')
bins_dzero = copy.copy(bins_zero)
bins_dzero.append('00')

So what's the deal with the zero and double zero? Why does it matter? Although it doesn't seem like much, it pushes the odds slightly in the house's favor. The payoffs remain the same, but your chances of winning go down.

+ even/odd strategy - the zero and double zero are not counted as even numbers. Your chance of winning is now 18/37 = 0.4865 (w/ zero) and 18/38 = 0.4739 (w/ double zero)
+ single number strategy - can choose any number, including zero or double zero, but odds of winning are now 1/37 or 1/38 instead of 1/36.

Before we get started with our game, let's generate some spins and convince ourselves that the wheel at least looks fair.

In [4]:
# nspins = number of spins
def spin(spins_num, bin_type):
    nspins = spins_num

    # Select a wheel (uncomment one of the following)
    #bins = bins_nozero
    #bins = bins_zero
    bins = bins_dzero

    spins = np.random.choice(bins, size=nspins)
    od = OrderedDict(sorted(Counter(spins).items()))
    print('Minimum count = ', min(od.values()))
    print('Maximum count = ', max(od.values()))
    print('Min/Max = ', min(od.values())/max(od.values()))

    labels = od.keys()
    counts = od.values()

    f, ax = plt.subplots(1, 1, figsize=(12,4))
    x_pos = np.arange(len(labels))
    ax.bar(x_pos, counts, align='center')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(labels)
    ax.set_ylabel('Roulette spins by bin value')
    ax.set_title('Roulette result counts')
    plt.show()
    
# Second function after inputed range. User chooses spins_num through slider and bin_type through dropdown
def spin_with_range(range):
    if range <= 100:
        interact(spin, spins_num=IntSlider(min=1,max=range,step=1,continuous_update=False), \
                 bin_type=[('bins_dzero', bins_dzero),('bins_zero', bins_zero), ('bins_nozero', bins_nozero)]
                )
    elif range <= 1000:
        interact(spin, spins_num=IntSlider(min=10,max=range,step=10,continuous_update=False), \
                 bin_type=[('bins_dzero', bins_dzero),('bins_zero', bins_zero), ('bins_nozero', bins_nozero)]
                )
    else:
        interact(spin, spins_num=IntSlider(min=1000,max=range,step=1000, continuous_update=False), \
                 bin_type=[('bins_dzero', bins_dzero),('bins_zero', bins_zero), ('bins_nozero', bins_nozero)],\
                 )
# User chooses the range of the number of spins        
interact(spin_with_range, range=[('100', 100), ('1000', 1000), ('100000', 100000), ('1000000', 1000000)]);

interactive(children=(Dropdown(description='range', options=(('100', 100), ('1000', 1000), ('100000', 100000),…

Now that we've setup our roulette wheel and convinced ourselves that it's working properly. Let's pretend that we're at the casino and see how our fortunes change over time. To simplify our code, we'll assume that you always select 'even' when using the even/odd strategy and '1' when selecting a single number. This is justified since all spins occur with equal probability.

In [5]:
# nspins = number of spins
def graph_roulette_spins(num_spins):
    nspins = num_spins

    # Strategies (uncomment one of the following)
    #strategy = 'even' # Win on 2, 4, 6, ... 36 but not 0 or 00
    strategy = 'single' # Win by picking any single number, including 0 or 00

    if strategy == 'even':
        payoff = 1  # Original bet plus 1
    elif strategy == 'single':
        payoff = 35 # original bet plus 35

    # Select a wheel (uncomment one of the following)
    #bins = bins_nozero
    #bins = bins_zero
    bins = bins_dzero

    balance = [0]
    spins = np.random.choice(bins, size=nspins)
    for spin in spins:
        if strategy == 'even':
            if int(spin) % 2 == 0 and int(spin) != 0:
                balance.append(balance[-1]+1)
            else:
                balance.append(balance[-1]-1)
        if strategy == 'single':
            if int(spin) == 1:
                balance.append(balance[-1]+35)
            else:
                balance.append(balance[-1]-1)

    if balance[-1] > 0:
        print("Win")
    elif balance[-1] < 0:
        print("Lose")
    else:
        print("Tie")

    x = range(nspins+1)
    xzero = [0, nspins]
    yzero = [0, 0]
        
    f, ax = plt.subplots(1, 1, figsize=(12,4))
    ax.plot(x, balance, xzero, yzero, '--')
    ax.set_ylabel('Balance')
    ax.set_xlabel('Spin number')
    ax.set_title('Balance as function of spins: strategy = ' + strategy)
    plt.show()
    
# Second function after range chosen. User-inputed number of spins
def spin_with_range(range):
    if range <= 100:
        interact(graph_roulette_spins, num_spins=IntSlider(min=1, max=range, step=1,continuous_update=False))
    elif range <= 1000:
        interact(graph_roulette_spins, num_spins=IntSlider(min=10, max=range, step=10,continuous_update=False))
    else:
        interact(graph_roulette_spins, num_spins=IntSlider(min=10000, max=range, step=10000,continuous_update=False))

# User chooses the range of the number of spins         
interact(spin_with_range, range=[('100', 100), ('1000', 1000), ('100000', 100000), ('1000000', 1000000)]);

interactive(children=(Dropdown(description='range', options=(('100', 100), ('1000', 1000), ('100000', 100000),…

See if there is a cleaner way to add a zero line to the graph